In [6]:
# 베이지안 최적화 - > optuna 사용할 거임
!pip install optuna

  Using cached optuna-3.6.1-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached Mako-1.3.3-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-3.6.1-py3-none-any.whl (380 kB)
   ---------------------------------------- 0.0/233.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/233.4 kB ? eta -:--:--
   ----- --------------------------------- 30.7/233.4 kB 435.7 kB/s eta 0:00:01
   ---------- ---------------------------- 61.4/233.4 kB 648.1 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/233.4 kB 521.8 kB/s eta 0:00:01
   ---------------------------------------- 233.4/233.4 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.8 kB ? eta -:--:--
   ---------------------------------------- 78.8/78.8 kB 4.6 MB/s eta 0:00:00


In [36]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


## optuna

In [39]:
import optuna
import plotly.express

In [9]:
def objective(trail) :
    x = trail.suggest_float("x",-10,10)
    return (x-2)**2


# 어떻게 학습할지에 대해 세팅
study = optuna.create_study()
study.optimize(objective, n_trials = 100)
study.best_params

[I 2024-05-09 13:44:13,291] A new study created in memory with name: no-name-a0e19661-b81a-42df-9012-e3b04c6e5626
[I 2024-05-09 13:44:13,294] Trial 0 finished with value: 1.7795218910296025 and parameters: {'x': 0.6660127845329242}. Best is trial 0 with value: 1.7795218910296025.
[I 2024-05-09 13:44:13,296] Trial 1 finished with value: 31.970582507234017 and parameters: {'x': 7.654253488059588}. Best is trial 0 with value: 1.7795218910296025.
[I 2024-05-09 13:44:13,297] Trial 2 finished with value: 2.3455695430991637 and parameters: {'x': 3.5315252342351933}. Best is trial 0 with value: 1.7795218910296025.
[I 2024-05-09 13:44:13,299] Trial 3 finished with value: 116.91295912735156 and parameters: {'x': -8.81262961204866}. Best is trial 0 with value: 1.7795218910296025.
[I 2024-05-09 13:44:13,300] Trial 4 finished with value: 84.29229183280862 and parameters: {'x': -7.181083369233101}. Best is trial 0 with value: 1.7795218910296025.
[I 2024-05-09 13:44:13,302] Trial 5 finished with valu

{'x': 2.0103510320718976}

참고) optuna에서 시도할 파라미터들에 대한 세팅  
==> titanic 데이터를 불러서 & 모델은 RF  
ref) https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial  

1) n_estimators : 정수값 : suggest_int(이름, 최소,최대값,,,)  
"n_estimators" : suggest_int("n_estimators", 10, 10000)  

2) criterion : "gini", "entropy", "log_loss"  
-> 종류 중 하나로 선택 :suggest_categorical  
"criterion" : suggest_categorical("criterion", ["gini", "entropy", "log_loss"])  

3) 수치범위인데 범위 내에서 균일분포. 로그 -> 비율에 대한 것을 지정할 때    
4) suggest_uniform, suggest_loguniform etc  
"min_samples_split" : suggest_uniform("min_samples_split", 0.2, 0.8)  

## data 불러오기

In [11]:
import pandas as pd
import numpy as np

# encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# divid data
from sklearn.model_selection import train_test_split

# validation 4개 중 선택
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold #-- 비율 유지 k-fold
from sklearn.model_selection import RepeatedKFold #-- 복원추출 k-fold
from sklearn.model_selection import RepeatedStratifiedKFold #-- 복원추출 비율 유지 k-fold

# 평가
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# 파라미터
from sklearn.model_selection import GridSearchCV  #--> 직접 다 체크
from sklearn.model_selection import RandomizedSearchCV #--> 랜덤하게 서치

# ML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [12]:
data = pd.read_csv("train.csv")

In [13]:
# 결측치 처리
def check_fillna(df):
    df.loc[:, "Age"].fillna(df.loc[:,"Age"].mean(), inplace=True)
    df.loc[:, "Cabin"].fillna("N", inplace = True)
    df.loc[:, "Embarked"].fillna("N", inplace = True)
    df.loc[:, "Fare"].fillna(0, inplace = True) #--> 결측치 대체 값은 변수유형도 확인할 것.

    return df


# 불필요한 컬럼 제거
def drop_feature(df):
    df.drop(["PassengerId","Name","Ticket"], axis=1, inplace = True)

    return df


# 인코딩 : 범주형은 숫자로 바꿔줘야함. (라벨인코딩: 1,2,3... / 원핫 인코딩 : 남,여 = 0,1 / 더미변수 : 남 = 0, 여 = 1)
def encode_fetures(df):
    # Cabin의 앞자리 1개만 가져올 거임. ex) C85 => "C"
    df.loc[:, "Cabin"] = df.loc[:, "Cabin"].apply(lambda x: str(x)[:1]) 

    #라벨 인코딩 
    cols = ["Cabin", "Sex", "Embarked"]

    for col in cols:
        le = LabelEncoder()
        le.fit(df.loc[:, col])

        df.isetitem( df.columns.get_loc(col), le.transform(df.loc[:, col]) ) #****

    return df

# 한 번에 표현
def titanic_preprocessing(df):
    df = check_fillna(df)
    df = drop_feature(df)
    df = encode_fetures(df)

    return df

In [14]:
data = titanic_preprocessing(data)


y_titanic = data.loc[:,"Survived"]
X_titanic = data.drop("Survived", axis = 1)


X_train, X_val, y_train, y_val = train_test_split(
    X_titanic,
    y_titanic,
    test_size = 0.2, #test 비율이 20%
    random_state = 1234, #꼭 1234 아니고 고정시키면 됨
    stratify = y_titanic
)

In [15]:
kfold = KFold( n_splits = 5, random_state=1234, shuffle=True)

## 랜덤포레스트

In [23]:
rf = RandomForestClassifier(n_jobs=-1, random_state=1234 )
scores = cross_val_score( rf,
                         X_train, y_train,
                          cv = kfold, scoring="accuracy")
scores

array([0.81118881, 0.84615385, 0.78873239, 0.76056338, 0.83098592])

In [24]:
# 이 다음에 최적의 RF을 찾기 위해서
# 오전에는 RGS방법을 통해서 범위를 좁힘

# 문제는 내가 선택하지 않은 값에서는 확인 불가능함!!!
# ==> optuna를 활용해서 최적의 파라미터를 찾아보자!!!

# 장점 : 했던 시행들이 종속적으로 서로 활용을 하면서 탐색!!
#        + 내가 전혀 생각하지 않은 값들에 대해서도 체크될 가능성.
# 단점 : 어느 시행의 횟수가 좀 확보가 되어야 함!!!

In [25]:
# optuna : 스스로 직접 목적함수를 세팅!!!!!!
#          ==> 정답은 없는데,,,본인 목적에 맞는 함수를 직접
#              만들 수 있어야 함...
def rf_objective( trail ) :
    # 1) Test를 할 파라미터들에 대한 가변적인 범위를 설정!!!
    paramas = {
        "n_estimators" : trail.suggest_int("n_estimators", 10,10000),
        "criterion" : trail.suggest_categorical("criterion", ["gini","entropy","log_loss"]),
        "max_features":trail.suggest_int("max_features", 1, X_train.shape[1]),
        "max_depth":trail.suggest_int("max_depth", 1, 50),
        "min_samples_split":trail.suggest_int("min_samples_split", 3,10)
    }
    # 2) 1번에 세팅한 파라미터들 받을 함수!!!!
    # ==> 직접 넘겨야 함!!!!!
    rf = RandomForestClassifier(n_jobs=-1, random_state=1234, **paramas )
    
    # 3-1) 그냥 통으로 학습한다면,,
    #rf.fit(X_train, y_train)
    # 3-2) CV를 한 결과를 바탕으로 하겠다.....
    scores = cross_val_score( rf,
                         X_train, y_train,
                        cv = kfold, scoring="accuracy")
    # --> cv=5에 대한 accuracy의 값이 존재!!!!
    
    # 저는 5번 시도 중에서 대표를 평균으로 하겠다..
    acc_mean = scores.mean()
    # 4) OPT를 할 값을 지정을 하게 됨!!!!!
    return acc_mean

In [26]:
# 위의 목적함수는 RF로 가려는 지향점을 설계!!!!!
# Next Step) 이 목적함수를 어떻게 학습할지 세팅!!!!!
# 예) 목적함수를 에러를 타켓: 학습의 방향을 min
# 예) 목적합수를 정확도 타켓 : 학습ㅇ의 방향을 max
rf_study = optuna.create_study(direction = "maximize")

# 공부의 방향성 : 내가 만든 목적함수의 target
# 실제
rf_study.optimize(rf_objective, n_trials = 50, n_jobs =-1)

# 참고) 위의 방식으로 하면 재현성이 없음. 중요하다고 생각하면 TPEsampler
# from optuna.samplers import TPESampler
# rf_study = optuna.create_study(direction = "maximize", sampler = TPESampler())

[I 2024-05-09 14:34:51,174] A new study created in memory with name: no-name-2fb3dbb8-4889-4b1b-b3d0-fedf6e4c7752
[I 2024-05-09 14:35:41,210] Trial 0 finished with value: 0.8089530188121739 and parameters: {'n_estimators': 382, 'criterion': 'gini', 'max_features': 1, 'max_depth': 45, 'min_samples_split': 9}. Best is trial 0 with value: 0.8089530188121739.
[I 2024-05-09 14:36:35,435] Trial 8 finished with value: 0.8271742342164877 and parameters: {'n_estimators': 644, 'criterion': 'log_loss', 'max_features': 5, 'max_depth': 26, 'min_samples_split': 9}. Best is trial 8 with value: 0.8271742342164877.
[I 2024-05-09 14:37:07,742] Trial 7 finished with value: 0.8215798286220821 and parameters: {'n_estimators': 2053, 'criterion': 'entropy', 'max_features': 3, 'max_depth': 3, 'min_samples_split': 10}. Best is trial 8 with value: 0.8271742342164877.
[I 2024-05-09 14:38:53,729] Trial 9 finished with value: 0.8187924751305033 and parameters: {'n_estimators': 1918, 'criterion': 'gini', 'max_featu

In [27]:
# 참고 : 50번은 좀 적게 한 부분이 있고,,
# ==> 설정한 파라미터들의 범위를 고려해서...

# 50번 시도 중에서 제일 cv평균이 높은 결과값
rf_study.best_trial.values

[0.828582684920713]

In [28]:
# 50번 시도 중에서 제일 좋았던 파라미터의 조합?
rf_study.best_params

{'n_estimators': 4560,
 'criterion': 'entropy',
 'max_features': 6,
 'max_depth': 6,
 'min_samples_split': 4}

In [30]:
# 단점은 할 때마다 다른 결과가 나옴!!!!!!

In [40]:
# + optuna 특징 중 하나가 시각화!!!!!!
# ==> 테스트한 결과들에 대한 시각적인 정보!!! plotly기본
optuna.visualization.plot_optimization_history(rf_study)